In [1]:
suppressPackageStartupMessages(library(BayesSpace))

In [21]:
data_dir <- "./data"
result_dir <- "./results"

In [ ]:
get_SingleCellExperiment <- function(path) {
    feature_file <- file.path(path, "features.tsv")
    observation_file <- file.path(path, "observations.tsv")
    matrix_file <- file.path(path, "counts.mtx")

    rowData <- read.delim(feature_file, stringsAsFactors = FALSE, row.names = 1)
    colData <- read.delim(observation_file, stringsAsFactors = FALSE, row.names = 1)
    counts <- as(Matrix::t(Matrix::readMM(matrix_file)), "CsparseMatrix")

    sce <- SingleCellExperiment(
        rowData = rowData,
        colData = colData,
        assays = list(counts = counts)
    )
    return(sce)
}

run_BayesSpace <- function(sce, n_clusters = 7, platform = "Visium", n_components = 15, n_HVG = 2000) {
    sce <- spatialPreprocess(
        sce,
        platform = platform,
        n.PCs = n_components,
        n.HVGs = n_HVG,
        log.normalize = TRUE
    )

    sce <- spatialCluster(
        sce,
        q = n_clusters,
        d = n_components,
        platform = platform
    )
    return(sce)
}

write_labels_to_tsv <- function(sce, path, col_name = "spatial.cluster") {
    label_df <- as.data.frame(colData(sce))[c(col_name)]

    dir.create(path, showWarnings = FALSE, recursive = TRUE)

    colnames(label_df) <- c("label")
    write.table(label_df, file = file.path(path, "BayesSpace.tsv"), sep = "\t", col.names = NA, quote = FALSE)
}

In [22]:
metadata <- read.delim(
    file.path(data_dir, "samples.tsv"),
    stringsAsFactors = FALSE,
    row.names = 1
)[, c("directory", "n_clusters")]

In [ ]:
for (i in 1:nrow(metadata)) {
    sample <- metadata[i, ]
    print(paste("Processing", sample[["directory"]]))
    sce <- get_SingleCellExperiment(file.path(data_dir, sample[["directory"]]))
    sce <- run_BayesSpace(sce, n_clusters = sample[["n_clusters"]])
    write_labels_to_tsv(sce, file.path(result_dir, sample[["directory"]]))
}

[1] "Processing Br5292_151507"


Neighbors were identified for 4226 out of 4226 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



[1] "Processing Br5292_151508"


Neighbors were identified for 4381 out of 4384 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



[1] "Processing Br5292_151509"


Neighbors were identified for 4789 out of 4789 spots.

Fitting model...

